In [1]:
import duckdb
import pandas as pd
import pyarrow.parquet as pq

In [2]:
conn = duckdb.connect(database=':memory:')

#### KM rodado

In [3]:
km_df = pd.DataFrame()

In [4]:
folder_path = '/home/marcelo-borges/Documentos/Projetos/datalakehouse_lowcost/elt/tmp/'

In [5]:
km_df=conn.execute(f""
                f"SELECT * FROM read_parquet('{folder_path}dna_kmRodado_*.parquet')"
                f"order by DATA, PREFIXO_CARRO").fetchdf()

In [6]:
km_df.columns

Index(['PREFIXO_CARRO', 'ID_EQUIPAMENTO', 'DATA', 'ID_LINHA', 'DS_TIPO_CHASSI',
       'DS_TIPO_MOTOR', 'DS_TIPO_CAMBIO', 'CARRO_AUXILIAR', 'FX_HORARIA',
       'FX_SIGLA', 'FX_NOME', 'KM_OCIOSA', 'LITROS_LINHA', 'KM_LINHA',
       'ABS_CARRO_KM_TOTAL_DIA', 'ABS_CARRO_LITROS_TOTAL_DIA'],
      dtype='object')

Entender o cenário onde pode ter 1 id_equipamento para mais de 1 prefixo_carro ou vice-versa. Porque em 1 ano isso não ocorreu e mesmo assim, quando forem analisar resultados sempre olharão em nível de prefixo de carro, minha sugestão é tirar o id_equipamento.

In [7]:
valores_unicos = km_df[['PREFIXO_CARRO', 'ID_EQUIPAMENTO']].drop_duplicates()

In [8]:
km_df.drop(columns=['ID_EQUIPAMENTO'], inplace=True)

In [9]:
carros = km_df[['PREFIXO_CARRO','DS_TIPO_CHASSI', 'DS_TIPO_CAMBIO','DS_TIPO_MOTOR','CARRO_AUXILIAR']].drop_duplicates()

In [10]:
carros

,PREFIXO_CARRO,DS_TIPO_CHASSI,DS_TIPO_CAMBIO,DS_TIPO_MOTOR,CARRO_AUXILIAR
0,20017,OF-1721,MANUAL,OM-924-LA.METÁLICO,NAO
13,20019,OF-1721,MANUAL,OM-924-LA.METÁLICO,NAO
22,20020,OF-1721,MANUAL,OM-924-LA.METÁLICO,NAO
27,20025,OF-1721,MANUAL,OM-924-LA.METÁLICO,NAO
32,20026,OF-1721,MANUAL,OM-924-LA.METÁLICO,NAO
...,...,...,...,...,...
539424,20961,SPRINTER,MANUAL,OM 651 CDI BI-TURBO,NAO
568585,20932,SPRINTER,MANUAL,OM 651 CDI BI-TURBO,NAO
582105,20942,SPRINTER,MANUAL,OM 651 CDI BI-TURBO,NAO
601637,20963,SPRINTER,MANUAL,OM 651 CDI BI-TURBO,NAO


In [11]:
carros.to_parquet('/home/marcelo-borges/Documentos/Projetos/datalakehouse_lowcost/elt/transmission/dimension/dim_carro.parquet')

In [12]:
km_df.drop(columns=['DS_TIPO_CHASSI', 'DS_TIPO_CAMBIO','CARRO_AUXILIAR','DS_TIPO_MOTOR'], inplace=True)

exclusão das colunas ABS_CARRO_KM_TOTAL E ABS_CARRO_LITROS, pois são valores repetidos obtidos pelo agrupamento das colunas LITROS_LINHA E KM_LINHA

In [13]:
km_df.drop(columns=['ABS_CARRO_KM_TOTAL_DIA', 'ABS_CARRO_LITROS_TOTAL_DIA'], inplace=True)

criação da tabela de horário e exclusão de informações redundantes

In [14]:
horarios = km_df[['FX_HORARIA','FX_SIGLA','FX_NOME']].drop_duplicates()

In [15]:
horarios.to_parquet('/home/marcelo-borges/Documentos/Projetos/datalakehouse_lowcost/elt/transmission/dimension/dim_horario.parquet')

In [16]:
km_df.drop(columns=['FX_SIGLA','FX_NOME'], inplace=True)

In [17]:
horarios

,FX_HORARIA,FX_SIGLA,FX_NOME
0,18,PT,4 PICO DA TARDE
1,15,EP,3 ENTRE PICO
2,14,EP,3 ENTRE PICO
3,11,EP,3 ENTRE PICO
4,6,PM,2 PICO DA MANHA
5,10,EP,3 ENTRE PICO
6,20,NT,5 NOTURNO
7,13,EP,3 ENTRE PICO
8,5,PM,2 PICO DA MANHA
9,9,EP,3 ENTRE PICO


casting

In [23]:
km_df

,PREFIXO_CARRO,DATA,ID_LINHA,FX_HORARIA,KM_OCIOSA,LITROS_LINHA,KM_LINHA
0,20017,01/01/2023,5274,18,"1,39773851590106007067137809187279151943","8,40282685512367491166077738515901060071","28,07773851590106007067137809187279151944"
1,20017,01/01/2023,5271,15,",8010278076509448455984022123214011368877","4,81556306652327546474112766938085727454","16,09102780765094484559840221232140113689"
2,20017,01/01/2023,5269,14,",8036472576432631740666769088953756337384","4,83131049316331233676447995083730219696","16,14364725764326317406667690889537563375"
3,20017,01/01/2023,5269,11,",8036472576432631740666769088953756337384","4,83131049316331233676447995083730219696","16,14364725764326317406667690889537563375"
4,20017,01/01/2023,5147,6,"1,41188354585957904440006145337225380243","8,48786295897987402058687970502381318175","28,36188354585957904440006145337225380244"
...,...,...,...,...,...,...,...
644537,20552,31/12/2023,5263,18,"6,99832784612718766245018194420377750823","13,54593657944896898284526078669208109513","42,14832784612718766245018194420377750821"
644538,20552,31/12/2023,5263,20,"6,99832784612718766245018194420377750823","13,54593657944896898284526078669208109513","42,14832784612718766245018194420377750821"
644539,20552,31/12/2023,5264,5,"7,32485011263212614798128573904002772483","14,17795182810604747877317622595737307226","44,11485011263212614798128573904002772483"
644540,20552,31/12/2023,5261,12,"5,48318488996707676312597470109166522267","10,61323167561947669381389707156472015249","33,02318488996707676312597470109166522268"


In [22]:
km_df.dtypes

PREFIXO_CARRO     int16
DATA             object
ID_LINHA         object
FX_HORARIA       object
KM_OCIOSA        object
LITROS_LINHA     object
KM_LINHA         object
dtype: object

In [52]:
km_df['DATA'] = pd.to_datetime(km_df['DATA'], format='%d/%m/%Y')
# km_df['DATA'] = km_df['DATA'].dt.strftime('%d/%m/%Y')

In [53]:
km_df['PREFIXO_CARRO'] = pd.to_numeric(km_df['PREFIXO_CARRO'],downcast='integer')

Transformando os dados de litros_linha que está em string para o formato númerico universal

In [54]:
km_df['LITROS_LINHA'] = km_df['LITROS_LINHA'].str.replace(',', '.')
km_df['LITROS_LINHA'] = pd.to_numeric(km_df['LITROS_LINHA'], downcast='float')

AttributeError: Can only use .str accessor with string values!

In [55]:
km_df['KM_OCIOSA'] = km_df['KM_OCIOSA'].str.replace(',', '.')
km_df['KM_OCIOSA'] = pd.to_numeric(km_df['KM_OCIOSA'], downcast='float')

AttributeError: Can only use .str accessor with string values!

In [56]:
km_df['KM_LINHA'] = km_df['KM_LINHA'].str.replace(',', '.')
km_df['KM_LINHA'] = pd.to_numeric(km_df['KM_LINHA'], downcast='float')

AttributeError: Can only use .str accessor with string values!

In [57]:
km_df.dtypes

PREFIXO_CARRO             int16
DATA             datetime64[ns]
ID_LINHA                  int16
FX_HORARIA               object
KM_OCIOSA               float32
LITROS_LINHA            float32
KM_LINHA                float32
dtype: object

Conclusão da tabela fato km_rodado

In [58]:
km_df.to_parquet('/home/marcelo-borges/Documentos/Projetos/datalakehouse_lowcost/elt/transmission/fact/fact_km_rodado.parquet')

#### Tabela dimensão de Linha

In [59]:
linha_df=conn.execute(f""
                f"SELECT * FROM read_parquet('{folder_path}dna_linha_*.parquet')"
                f"order by DATA_LINHA").fetchdf()

In [60]:
linha_df.head(5)

,ID_LINHA_BI,DATA_LINHA,DIA,MES,ANO,ID_EMPRESA,ID_LINHA,LINHA_NUM,LINHA_ALFA,DESCRICAO_LINHA,TIPO_LINHA,LOCAL_DE_CONTROLE,CODIGO_UN,NOME_UN,CODIGO_UGB,NOME_UGB,CODIGO_TN,NOME_TN
0,2023010125206,01/01/2023,01,01,2023,2,5206,941,A,T. CRUZEIRO/PCA TAMANDARE/EIXO T-8,TRONCAL,T. CRUZEIRO,004,SUL,0941,T. CRUZEIRO/PÇA TAMANDARÉ/EIXO T-8,01,SERVIÇO CONVENCIONAL ( ESSENCIAL )
1,2023010125341,01/01/2023,01,01,2023,2,5341,9,A,T. PE. PELAGIO / AV. CASTELO BRANCO / T. ISIDORIA,TRONCAL,TERMINAL PADRE PELAGIO,005,SUDESTE,0009,T. ISIDÓRIA / T. PADRE PELÁGIO,01,SERVIÇO CONVENCIONAL ( ESSENCIAL )
2,2023010125069,01/01/2023,01,01,2023,2,5069,3,I,T. MARANATA / T. BANDEIRAS,TRONCAL,T. MARANATA,001,SUDOESTE,0003-A,TERMINAL MARANATA / RODOVIARIA - EIXO T-7,01,SERVIÇO CONVENCIONAL ( ESSENCIAL )
3,2023010125070,01/01/2023,01,01,2023,2,5070,4,A,T. GARAVELO / CENTRO - EIXO T-09,TRONCAL,T. GARAVELO,001,SUDOESTE,0004-A,TERMINAL GARAVELO / CENTRO - EIXO T-9,01,SERVIÇO CONVENCIONAL ( ESSENCIAL )
4,2023010123601,01/01/2023,01,01,2023,2,3601,826,,CATRACA DE SOLO T. GARAVELO,CATRACA DE SOLO,T. GARAVELO,001,SUDOESTE,0826,CATRACA DE SOLO TERMINAL GARAVELO,01,SERVIÇO CONVENCIONAL ( ESSENCIAL )


exclusão de todas as colunas que não são pertinentes a dimensão linha

In [61]:
linha_df.drop(columns=['ID_LINHA_BI','DATA_LINHA','DIA','MES','ANO','ID_EMPRESA','CODIGO_TN','NOME_TN'], inplace=True)

In [62]:
linha_df.drop_duplicates(inplace=True)

In [63]:
linha_df.to_parquet('/home/marcelo-borges/Documentos/Projetos/datalakehouse_lowcost/elt/transmission/dimension/dim_linha.parquet')

#### Tabela dimensão de Calendário

In [98]:
folder_path = '/home/marcelo-borges/Documentos/Projetos/datalakehouse_lowcost/elt/tmp/'

In [99]:
calendar_df=conn.execute(f""
                f"SELECT * FROM read_parquet('{folder_path}dna_calendario_*.parquet')"
                f"order by ID_CALENDARIO ").fetchdf()

In [100]:
calendar_df.drop(columns=['ID_EMPRESA', 'NOME_EMPRESA','SIGLA_EMPRESA'], inplace=True)

casting

In [101]:
calendar_df['DATA'] = pd.to_datetime(calendar_df['DATA'], format='%d/%m/%Y')
# calendar_df['DATA'] = calendar_df['DATA'].dt.strftime('%d/%m/%Y')

In [102]:
calendar_df['DIA'] = calendar_df['DATA'].dt.day
calendar_df['MES'] = calendar_df['DATA'].dt.month
calendar_df['ANO'] = calendar_df['DATA'].dt.year

In [107]:
calendar_df.dtypes

DATA                  datetime64[ns]
ID_CALENDARIO                  int16
ID_OPERACAO_TIPO                int8
NOME_OPERACAO_TIPO            object
DIA                            int32
MES                            int32
ANO                            int32
dtype: object

casting

In [106]:
calendar_df['ID_CALENDARIO'] = pd.to_numeric(calendar_df['ID_CALENDARIO'], downcast='integer')
calendar_df['ID_OPERACAO_TIPO'] = pd.to_numeric(calendar_df['ID_OPERACAO_TIPO'], downcast='integer')

In [108]:
calendar_df.to_parquet('/home/marcelo-borges/Documentos/Projetos/datalakehouse_lowcost/elt/transmission/dimension/dim_calendar.parquet')